w_03 | BeautifulSoup
====

BeautifulSoup es, junto con Scrapy, el paquete más usado para scrapping web. Su uso básico tiene una lógica muy simple que consiste en proveerle el código fuente de una página (el HTML) para que construya un "árbol" con sus elementos en donde podemos buscar lo que necesitamos.

Busquemos todas las noticias de la portada de Montevideo Portal, para lo que hay que inspeccionar la fuente.

![](https://drive.google.com/uc?export=view&id=1ypYmHKOQX281hqE5aLv0J39s0-oP56A4)

In [1]:
from pprint import pprint

import requests
import pandas as pd
from bs4 import BeautifulSoup

url = "https://www.montevideo.com.uy/categoria/Noticias-310"
r = requests.get(url)
soup = BeautifulSoup(r.content)
find = soup.find_all("h2", class_="title")
results = {el.contents[0].text: el.contents[0].get("href") for el in find}
pprint(results, indent=4)

{   'Argentina suspendió por 30 días las exportaciones de carne para contener el precio interno': 'https://www.montevideo.com.uy/Noticias/Argentina-suspendio-por-30-dias-las-exportaciones-de-carne-para-contener-el-precio-interno-uc786772',
    'Biden expresa su apoyo a un alto el fuego durante una llamada con Netanyahu': 'https://www.montevideo.com.uy/Noticias/Biden-expresa-su-apoyo-a-un-alto-el-fuego-durante-una-llamada-con-Netanyahu-uc786777',
    'Bomberos extinguió incendio en el histórico teatro 28 de Febrero de Mercedes, en Soriano': 'https://www.montevideo.com.uy/Noticias/Bomberos-extinguio-incendio-en-el-historico-teatro-28-de-Febrero-de-Mercedes-en-Soriano-uc786770',
    'Duque ordena desplegar a la fuerza pública para desbloquear vías en Colombia': 'https://www.montevideo.com.uy/Noticias/Duque-ordena-desplegar-a-la-fuerza-publica-para-desbloquear-vias-en-Colombia-uc786778',
    'Hubo 62 fallecimientos y se registraron 2.399 casos nuevos de covid en 11.184 análisis': 'https://

Hicimos lo siguiente:

1. Un request para obtener el HTML.
2. Pasamos el contenido de la respuesta a la clase ``BeautifulSoup`` (se llama igual que el paquete, pero se importa de ``bs4``), lo que devuelve un objeto que podemos procesar fácilmente.
3. Usamos el método ``find_all()`` para que encuentre todos los elementos con el tag h2 y la class "title", porque eso es lo que figura cuando hacemos inspect en un browser.
4. El texto y el link están un nivel más abajo del tag h2, por lo que tenemos que acceder a los ``contents`` (es una lista, por eso accedemos al elemento 0, el único que hay en este caso).
5. Armamos una dict comprehension que extrae el texto y el link (href) de cada elemento encontrado.

¿Por qué no buscamos directamente los tags a (que son links)? Porque necesitamos alguna forma de "filtrar" los resultados, y como ese tag no tiene demasiados detalles, recurrimos a su parent h2. Sino, pasa lo siguiente.

In [2]:
soup.find_all("a")[:10]

[<a class="icon notificaciones" href="" onclick="return false;" target="_blank" title="Notificaciones"></a>,
 <a class="later notificationsReminder" href="#">Recordar más tarde</a>,
 <a class="icon facebook" href="https://www.facebook.com/portalmvd" target="_blank" title="Facebook"></a>,
 <a class="icon twitter" href="https://twitter.com/portalmvd" target="_blank" title="Twitter"></a>,
 <a class="icon instagram" href="https://www.instagram.com/portalmvd/" target="_blank" title="Instagram"></a>,
 <a class="icon youtube" href="https://www.youtube.com/c/MontevideoPortalVideos" target="_blank" title="Youtube"></a>,
 <a class="icon spotify" href="https://open.spotify.com/show/7qCGYvcXw1WzqRH5I5UZfh" target="_blank" title="Spotify"></a>,
 <a class="icon rss" href="https://www.montevideo.com.uy/auc.aspx?27383" target="_blank" title="RSS"></a>,
 <a class="icon itunes" href="https://itunes.apple.com/app/id400466279" target="_blank" title="iTunes"></a>,
 <a class="icon android" href="https://pla

Con el método ``find_all()`` y un poco de creatividad se puede hacer prácticamente todo.

La idea ahora es extraer avisos de puestos de trabajo de El Gallito. En la URL de búsqueda hay información limitada, por lo que habrá que entrar uno por uno, para lo cual necesitamos extraer las URLs.

Vamos a concentranos en los resultados de la primera página.

El proceso es igual, inspeccionamos el elemento que nos interesa, ubicamos los tags relevantes y extraemos.

In [3]:
gallito = "https://trabajo.gallito.com.uy/buscar"
r = requests.get(gallito)
soup = BeautifulSoup(r.content)
find = soup.find_all("a", class_="post-cuadro row smB")
urls = [puesto.get("href") for puesto in find]
urls

['/anuncio/administrativo-contable-h4dj9',
 '/anuncio/fonoaudiologo-a-hnz2e',
 '/anuncio/vendedor-distribuidor-freelance-n9szv',
 '/anuncio/ayudante-de-panaderia-avanzado-qm2r9',
 '/anuncio/limpiador-a-para-panaderia-6arrb',
 '/anuncio/vendedores-part-time-full-time-1-ezq5b',
 '/anuncio/vendedores-as-2-patz3',
 '/anuncio/cuidador-de-las-personas-mayores-pm-en-residenciales-mu98z',
 '/anuncio/independiente-8t7gq',
 '/anuncio/responsable-de-administracion-ra5v9',
 '/anuncio/oficial-electricista-4mv8j',
 '/anuncio/cobrador-con-moto-para-area-metropolitana-77sg5',
 '/anuncio/cortador-srb2j',
 '/anuncio/c-bjb6y',
 '/anuncio/vendedor-de-calle-con-moto-para-costa-de-oro-vm3mq',
 '/anuncio/ortodoncista-5gtsm',
 '/anuncio/agente-de-ventas-online-teletrabajo-6a3re',
 '/anuncio/peon-v7rss',
 '/anuncio/acompanante-con-cama-n6jdy',
 '/anuncio/docente-profesor-9jdm9']

La tarea ahora es constuir las URLs, hacer el request y luego el scrapping.

La ventaja de HTML es que suele estar todo en el mismo lugar. Entonces con ver la estructura en un puesto podemos saber cómo van a funcionar el resto.

In [4]:
puestos = []
with requests.Session() as s:
    for url in urls:
        full_url = f"https://trabajo.gallito.com.uy{url}"
        print(f"Procesando {full_url}")
        r = s.get(full_url)
        soup = BeautifulSoup(r.content)
        title = soup.find("div", class_="title-puesto").contents[0].text.strip()
        subtitle = soup.find("div", class_="subtitle-puesto").text.strip()
        since = soup.find("span", class_="time-text").text.strip()
        cantidad = soup.find("div", class_="span-ofertas").text.strip()
        titulo_textos = soup.find_all("div", class_="cuadro-aviso-title")
        textos = soup.find_all("div", class_="cuadro-aviso-text")
        textos = {k.text.strip(): v.text.strip() for k, v in zip(titulo_textos, textos)}
        textos.pop("Avisos similares", None)
        data_puesto = {"Descripción": title, "Empresa": subtitle, "Cantidad de ofertas": cantidad, "Fecha": since}
        data_puesto.update(textos)
        puestos.append(data_puesto)

df = pd.DataFrame(puestos)
df.head()

Procesando https://trabajo.gallito.com.uy/anuncio/administrativo-contable-h4dj9
Procesando https://trabajo.gallito.com.uy/anuncio/fonoaudiologo-a-hnz2e
Procesando https://trabajo.gallito.com.uy/anuncio/vendedor-distribuidor-freelance-n9szv
Procesando https://trabajo.gallito.com.uy/anuncio/ayudante-de-panaderia-avanzado-qm2r9
Procesando https://trabajo.gallito.com.uy/anuncio/limpiador-a-para-panaderia-6arrb
Procesando https://trabajo.gallito.com.uy/anuncio/vendedores-part-time-full-time-1-ezq5b
Procesando https://trabajo.gallito.com.uy/anuncio/vendedores-as-2-patz3
Procesando https://trabajo.gallito.com.uy/anuncio/cuidador-de-las-personas-mayores-pm-en-residenciales-mu98z
Procesando https://trabajo.gallito.com.uy/anuncio/independiente-8t7gq
Procesando https://trabajo.gallito.com.uy/anuncio/responsable-de-administracion-ra5v9
Procesando https://trabajo.gallito.com.uy/anuncio/oficial-electricista-4mv8j
Procesando https://trabajo.gallito.com.uy/anuncio/cobrador-con-moto-para-area-metropoli

,Descripción,Empresa,Cantidad de ofertas,Fecha,Responsabilidades,Funciones,Requisitos
0,ADMINISTRATIVO CONTABLE,,1 ofertas laborales,Hace 4 horas,Liquidación de haberes.,Administrativo contable con conocimientos y e...,Experiencia de 1 año en el área de Recursos hu...
1,FONOAUDIÓLOGO/A,Consultorio privado,2 ofertas laborales,Hace 9 horas,"Asistencia, y valoración alteraciones del leng...","Evaluación, asesoramiento, tratamientos",NaN
2,VENDEDOR DISTRIBUIDOR FREELANCE,Diamante del Este,1 ofertas laborales,Hace 9 horas,Ajustarse a las normas comerciales del proveed...,Ventas directas y Distribución independiente\n...,NaN
3,AYUDANTE DE PANADERÍA AVANZADO,Empresa,2 ofertas laborales,Hace 1 día,.,"Seleccionamos Ayudante de Panadería avanzado, ...",NaN
4,LIMPIADOR/A PARA PANADERIA,Empresa,2 ofertas laborales,Hace 1 día,.,Seleccionamos limpiador o limpiadora para Pana...,NaN


Algunos comentarios:

1. Usamos una ``Session`` de ``requests`` para agilizar el proceso dado que estamos entrando a varias páginas del mismo sitio.
2. Para el título tenemos que revisar los ``contents``, porque es un h2 dentro de un div. No podemos buscar directamente los h2 porque hay otros títulos de ese tipo en la página.
3. El subtítulo (que tiene la información de la empresa), la cantidad de ofertas y el momento de publicación no tienen dificultades, simplemente accedemos a su ``text`` y le aplicamos el método ``strip()`` para que saque todos los espacios extra.
4. Cada puesto tiene distintos boxes con información que no están presentes en todos los avisos. Sin embargo, siempre tienen los mismos tags. Por eso  hacemos un ``find_all()`` para esos tags y luego lo procesamos con una dict comprehension.
5. Algunos avisos tienen un box de "Avisos similares" que no nos interesa por ahora, así que lo eliminamos del diccionario de textos.
6. Construimos un nuevo diccionario con el título, subtítulo, cantidad de ofertas y momento de publicación.
7. Actualizamos este diccionario con la información de los textos.
8. Agregamos el diccionario completo a la lista de ``puestos``